# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [3]:
bootstrap_idx(X_train)

array([ 37,  65,  12,  86,  30,  71,  15,  79,  75,  49,   5, 101,  76,
         2,  84,  69,  10,  57, 107,   8,  25,  73,  47, 103,  89,   8,
        47, 107,  72,  46,  31,  41,  25,  78,   6,  79,   3, 109,  27,
        66,  26,  55,  37,  40,  95,  59,  12,  35,  73,  23,  10, 109,
        12,  71,  32,  89, 110,  65,   4, 100,  20,  89,  36,  61, 106,
         5,  99,  48,  45,  26,  35,  46,   9,   9,  98,  16,  99,  84,
        81,  31,  74,  20,  58,  67,  99,  11,  28,  79,  88,  95,  99,
        58,  67,  65,   9,   4,  39,  51,   4,  92,  63,  27,  11,  24,
        82,   6,  79,  50,  50,  25,  53,  95])

In [4]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({28: 4,
         48: 3,
         50: 3,
         54: 3,
         26: 3,
         34: 3,
         65: 3,
         18: 3,
         57: 3,
         105: 2,
         35: 2,
         24: 2,
         96: 2,
         94: 2,
         67: 2,
         84: 2,
         36: 2,
         31: 2,
         49: 2,
         66: 2,
         5: 2,
         37: 2,
         82: 2,
         97: 2,
         71: 2,
         11: 2,
         74: 2,
         107: 2,
         87: 2,
         13: 1,
         52: 1,
         33: 1,
         81: 1,
         53: 1,
         83: 1,
         46: 1,
         92: 1,
         70: 1,
         23: 1,
         17: 1,
         39: 1,
         77: 1,
         98: 1,
         68: 1,
         80: 1,
         111: 1,
         91: 1,
         64: 1,
         43: 1,
         79: 1,
         14: 1,
         20: 1,
         104: 1,
         78: 1,
         72: 1,
         59: 1,
         93: 1,
         75: 1,
         15: 1,
         76: 1,
         22: 1,
         32: 1,
     

In [5]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [6]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [7]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({1: 38, 0: 37, 2: 37})
Classes distribution in the bootstrap: Counter({0: 44, 1: 41, 2: 27})


<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [10]:
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import mode
from sklearn.metrics import accuracy_score

ntrees = 5
max_depth = 3

tree_list = []
ypred_list = []
for i in range(ntrees):
    X_train_i, y_train_i = bootstrap_sample(X_train, y_train)
    treei = DecisionTreeClassifier(max_depth=max_depth)
    treei.fit(X_train_i, y_train_i)
    ypred_list.append(treei.predict(X_test))
    tree_list.append(treei)
    
ypred,_ = mode(np.array(ypred_list), axis=0, keepdims=False)
print(accuracy_score(y_test, ypred))

0.9736842105263158


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [13]:
from sklearn.ensemble import BaggingClassifier

bagging = BaggingClassifier(
    estimator=DecisionTreeClassifier(max_depth=max_depth),
    n_estimators=ntrees
)
bagging.fit(X_train, y_train)
print(bagging.score(X_test, y_test))

0.9736842105263158


## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [ ]:
from figures import plot_forest_interactive
plot_forest_interactive()